In [1]:
# ref https://towardsdatascience.com/applying-machine-learning-to-classify-an-unsupervised-text-document-e7bb6265f52

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from matplotlib import figure
%matplotlib inline  
import math
import nltk
from nltk.corpus import stopwords

import pandas as pd
from scipy.sparse import coo_matrix

from operator import itemgetter
import heapq
import collections

doclist = []
path = 'Data/1800FLOWERSCOMInc.txt'
path1 = 'Data/2UInc.txt'
path2 = 'Data/3DSystemsCorporation.txt'
paths = [path, path1, path2]
for path in paths:
    file = open(path, "r") 
    # print "Output of Readlines after appending"
    text = file.readlines()
    doclist.append(text[0])


In [20]:
def split_document(doc_text):
    split_text = doc_text.split(".")
    vectorizer = TfidfVectorizer(stop_words='english')
    # TODO: Custom stopwords, 1-800-flowers returned in multiple slices/formats-> Replace with [name]?
    X = vectorizer.fit_transform(split_text)
    # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer.fit_transform
    return X

In [21]:
doclist[0].split(".")

['[1] "Item 1A',
 ' Risk Factors',
 ' Cautionary Statements Under the Private Securities Litigation Reform Act of 1995',
 ' Our disclosures and analysis in this Form 10-K contain some forward-looking statements that set forth anticipated results based on management’s plans and assumptions',
 ' From time to time, we also provide forward-looking statements in other statements we release to the public as well as oral forward-looking statements',
 ' Such statements give our current expectations or forecasts of future events; they do not relate strictly to historical or current facts',
 ' We have tried, wherever possible, to identify such statements by using words such as “anticipate,” “estimate,” “expect,” “project,” “intend,” “plan,” “believe” and similar expressions in connection with any discussion of future operating or financial performance',
 ' In particular, these include statements relating to future actions; the effectiveness of our marketing programs; the performance of our exist

In [22]:
X = split_document(doclist[0])
X

<337x1161 sparse matrix of type '<class 'numpy.float64'>'
	with 3211 stored elements in Compressed Sparse Row format>

In [23]:

# https://stackoverflow.com/questions/22961541/python-matplotlib-plot-sparse-matrix-pattern
def plot_coo_matrix(m):
    if not isinstance(m, coo_matrix):
        m = coo_matrix(m)
    fig = pyplot.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, facecolor ='black')
    ax.plot(m.col, m.row, 's', color='white', ms=3)
    ax.set_xlim(0, m.shape[1])
    ax.set_ylim(0, m.shape[0])
    ax.set_aspect('equal')
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    return ax



ax = plot_coo_matrix(X)
ax.figure.show()


NameError: name 'pyplot' is not defined

In [24]:

# truek = PLEASE_FIND_ME
truek = 25
model = KMeans(n_clusters=truek, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=25, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [25]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

NameError: name 'vectorizer' is not defined

In [ ]:
nclosest_words_to_show = 5
for i in range(truek):
 print('Cluster %d:' % i),
 for ind in order_centroids[i, :nclosest_words_to_show]:
     print(' %s' % terms[ind])

In [ ]:
print('\n')
print('Prediction')
X = vectorizer.transform(['Market outlook on coffee looks promising'])
predicted = model.predict(X)
print(predicted)


In [ ]:
# HOW DO WE FIND OUT WHAT #7 meams?, 1Closest_word to cluster definitely isn't it
for i in range(truek):
 print('Cluster %d:' % i),
 for ind in order_centroids[i, :1]:
     print(' %s' % terms[ind])

In [ ]:
# using joblib to pickle model
from sklearn.externals import joblib
joblib.dump(model,  'doc_cluster.pkl')
model = joblib.load('doc_cluster.pkl')

In [26]:
## TF-IDF analysis 
def get_wordset(doc_text):
    words = doc_text.split(" ")
    wordSet = []
    stopWords = set(stopwords.words('english'))
    print(stopWords)
    for w in words:
    #     wordSet.append(w)

        if w not in stopWords:
            wordSet.append(w)
    return wordSet

In [27]:
def get_wordDict(wordSet):
    wordDict = dict.fromkeys(wordSet, 0) 
    for word in wordSet:
        wordDict[word]+=1

    return wordDict

In [28]:
def computeTF(document_text):
    wordset = get_wordset(document_text)
    word_dict = get_wordDict(wordset)
    tfDict = {}
    word_count = len(wordset)
    for word, count in word_dict.items():
        tfDict[word] = count/float(word_count)
    return tfDict

In [29]:
term_frequency = computeTF(doclist[0])

{'once', 'here', 'any', 'she', 'most', 'weren', 'his', 'of', 'through', 'does', 'further', "shouldn't", 'same', 'what', 'have', 'ain', 'down', 'by', 'but', 'then', 'between', 'over', 'will', 'didn', "mightn't", "won't", 'myself', 'against', 'doing', 'shan', 'while', 'in', 'about', 'a', 'for', 'as', 'yourself', 'aren', 'both', 'now', 'be', 'were', 'being', 'why', "wouldn't", 'are', 'other', 'own', "wasn't", 'd', 'my', 'above', 'i', 'ours', 'at', 'which', "that'll", 'the', 'all', 'when', "hadn't", 'because', 'very', 'your', "doesn't", 'you', "needn't", 'haven', 'don', "she's", 'can', 'an', 'again', "you've", 'himself', 'before', 'was', 'been', 's', 'll', 'whom', 'too', 'them', 'these', 'needn', 'few', 'each', 'me', 'until', 'he', 'wouldn', 'just', 'who', 'than', 'or', 'if', 'do', 'is', 'itself', 'more', 'hadn', 've', "hasn't", 'o', 'ourselves', 'having', 'mightn', "aren't", 'below', "shan't", 'shouldn', 'how', 'this', 'couldn', 'with', 'has', 'doesn', 'isn', 'some', 'our', "you're", 'off

In [30]:
term_frequency

{'[1]': 0.0002536783358701167,
 '"Item': 0.0002536783358701167,
 '1A.': 0.0002536783358701167,
 'Risk': 0.0002536783358701167,
 'Factors.': 0.0002536783358701167,
 'Cautionary': 0.0002536783358701167,
 'Statements': 0.0002536783358701167,
 'Under': 0.0005073566717402334,
 'Private': 0.0005073566717402334,
 'Securities': 0.0010147133434804667,
 'Litigation': 0.00076103500761035,
 'Reform': 0.0005073566717402334,
 'Act': 0.00076103500761035,
 '1995.': 0.0002536783358701167,
 'Our': 0.0012683916793505834,
 'disclosures': 0.0005073566717402334,
 'analysis': 0.0002536783358701167,
 'Form': 0.0015220700152207,
 '10-K': 0.0005073566717402334,
 'contain': 0.00076103500761035,
 'forward-looking': 0.0012683916793505834,
 'statements': 0.0015220700152207,
 'set': 0.0002536783358701167,
 'forth': 0.0002536783358701167,
 'anticipated': 0.0002536783358701167,
 'results': 0.00837138508371385,
 'based': 0.0002536783358701167,
 'management’s': 0.0005073566717402334,
 'plans': 0.0005073566717402334,
 'a

In [31]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    big_wordset = []
    for doc in docList:
        big_wordset += get_wordset(doc)
    big_wordDict = get_wordDict(big_wordset)
        
    idfDict = dict.fromkeys(big_wordDict.keys(), 0)
  
    for word, val in big_wordDict.items():
        if val > 0:
            idfDict[word] += 1

    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [32]:
idfs = computeIDF(doclist)
idfs

{'once', 'here', 'any', 'she', 'most', 'weren', 'his', 'of', 'through', 'does', 'further', "shouldn't", 'same', 'what', 'have', 'ain', 'down', 'by', 'but', 'then', 'between', 'over', 'will', 'didn', "mightn't", "won't", 'myself', 'against', 'doing', 'shan', 'while', 'in', 'about', 'a', 'for', 'as', 'yourself', 'aren', 'both', 'now', 'be', 'were', 'being', 'why', "wouldn't", 'are', 'other', 'own', "wasn't", 'd', 'my', 'above', 'i', 'ours', 'at', 'which', "that'll", 'the', 'all', 'when', "hadn't", 'because', 'very', 'your', "doesn't", 'you', "needn't", 'haven', 'don', "she's", 'can', 'an', 'again', "you've", 'himself', 'before', 'was', 'been', 's', 'll', 'whom', 'too', 'them', 'these', 'needn', 'few', 'each', 'me', 'until', 'he', 'wouldn', 'just', 'who', 'than', 'or', 'if', 'do', 'is', 'itself', 'more', 'hadn', 've', "hasn't", 'o', 'ourselves', 'having', 'mightn', "aren't", 'below', "shan't", 'shouldn', 'how', 'this', 'couldn', 'with', 'has', 'doesn', 'isn', 'some', 'our', "you're", 'off

{'[1]': 0.47712125471966244,
 '"Item': 0.47712125471966244,
 '1A.': 0.47712125471966244,
 'Risk': 0.47712125471966244,
 'Factors.': 0.47712125471966244,
 'Cautionary': 0.47712125471966244,
 'Statements': 0.47712125471966244,
 'Under': 0.47712125471966244,
 'Private': 0.47712125471966244,
 'Securities': 0.47712125471966244,
 'Litigation': 0.47712125471966244,
 'Reform': 0.47712125471966244,
 'Act': 0.47712125471966244,
 '1995.': 0.47712125471966244,
 'Our': 0.47712125471966244,
 'disclosures': 0.47712125471966244,
 'analysis': 0.47712125471966244,
 'Form': 0.47712125471966244,
 '10-K': 0.47712125471966244,
 'contain': 0.47712125471966244,
 'forward-looking': 0.47712125471966244,
 'statements': 0.47712125471966244,
 'set': 0.47712125471966244,
 'forth': 0.47712125471966244,
 'anticipated': 0.47712125471966244,
 'results': 0.47712125471966244,
 'based': 0.47712125471966244,
 'management’s': 0.47712125471966244,
 'plans': 0.47712125471966244,
 'assumptions.': 0.47712125471966244,
 'From': 

In [33]:
def computeTFIDF(term_frequency, idfs):
    tfidf = {}
    for word, val in term_frequency.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [34]:
tfidfBow = computeTFIDF(term_frequency, idfs)


In [35]:
tfidfBow


{'[1]': 0.00012103532590554602,
 '"Item': 0.00012103532590554602,
 '1A.': 0.00012103532590554602,
 'Risk': 0.00012103532590554602,
 'Factors.': 0.00012103532590554602,
 'Cautionary': 0.00012103532590554602,
 'Statements': 0.00012103532590554602,
 'Under': 0.00024207065181109204,
 'Private': 0.00024207065181109204,
 'Securities': 0.0004841413036221841,
 'Litigation': 0.0003631059777166381,
 'Reform': 0.00024207065181109204,
 'Act': 0.0003631059777166381,
 '1995.': 0.00012103532590554602,
 'Our': 0.0006051766295277301,
 'disclosures': 0.00024207065181109204,
 'analysis': 0.00012103532590554602,
 'Form': 0.0007262119554332762,
 '10-K': 0.00024207065181109204,
 'contain': 0.0003631059777166381,
 'forward-looking': 0.0006051766295277301,
 'statements': 0.0007262119554332762,
 'set': 0.00012103532590554602,
 'forth': 0.00012103532590554602,
 'anticipated': 0.00012103532590554602,
 'results': 0.003994165754883019,
 'based': 0.00012103532590554602,
 'management’s': 0.00024207065181109204,
 'pl

In [36]:
# from https://stackoverflow.com/questions/4743035/obtaining-the-least-common-element-in-array
def least_common_values(array, to_find=None):
    counter = collections.Counter(array)
    if to_find is None:
        return sorted(counter.items(), key=itemgetter(1), reverse=False)
    return heapq.nsmallest(to_find, counter.items(), key=itemgetter(1))
least_common_values(tfidfBow, 5)

[('[1]', 0.00012103532590554602),
 ('"Item', 0.00012103532590554602),
 ('1A.', 0.00012103532590554602),
 ('Risk', 0.00012103532590554602),
 ('Factors.', 0.00012103532590554602)]

In [37]:
collections.Counter(tfidfBow).most_common(20)


[('may', 0.014403203782759976),
 ('Company’s', 0.010530073353782504),
 ('Company', 0.010409038027876958),
 ('The', 0.006777978250710577),
 ('could', 0.006656942924805031),
 ('customers', 0.0043572717325996565),
 ('results', 0.003994165754883019),
 ('If', 0.0037520951030719264),
 ('<U+25CF>', 0.0031469184735441963),
 ('\\t\\t\\t', 0.0031469184735441963),
 ('products', 0.0030258831476386505),
 ('customer', 0.0029048478217331046),
 ('significant', 0.0022996711922053745),
 ('In', 0.0021786358662998283),
 ('ability', 0.0021786358662998283),
 ('use', 0.0021786358662998283),
 ('operating', 0.0019365652144887364),
 ('result', 0.0019365652144887364),
 ('number', 0.0019365652144887364),
 ('affect', 0.0019365652144887364)]

In [38]:
collections.Counter(term_frequency).most_common(20)



[('may', 0.030187721968543885),
 ('Company’s', 0.02207001522070015),
 ('Company', 0.021816336884830034),
 ('The', 0.014205986808726534),
 ('could', 0.013952308472856417),
 ('customers', 0.0091324200913242),
 ('results', 0.00837138508371385),
 ('If', 0.007864028411973617),
 ('<U+25CF>', 0.006595636732623034),
 ('\\t\\t\\t', 0.006595636732623034),
 ('products', 0.006341958396752917),
 ('customer', 0.0060882800608828),
 ('significant', 0.004819888381532217),
 ('In', 0.0045662100456621),
 ('ability', 0.0045662100456621),
 ('use', 0.0045662100456621),
 ('operating', 0.004058853373921867),
 ('result', 0.004058853373921867),
 ('number', 0.004058853373921867),
 ('affect', 0.004058853373921867)]

In [39]:
# Latent Dirichlet Allocation
# Ref http://brandonrose.org/clustering
# LDA is a probabilistic topic model that assumes documents
# are a mixture of topics and that each word in the document is attributable to the document's topics.
from nltk.stem.snowball import SnowballStemmer
import re
stemmer = SnowballStemmer("english")
import nltk
nltk.download('stopwords')
 
stopwords = nltk.corpus.stopwords.words('english')

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns
from gensim import corpora, models, similarities 

#remove proper names0
%time preprocess = [strip_proppers(doc) for doc in input_texts]

#tokenize
%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]

#remove stop words
%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]
# print(texts)
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)
# print(dictionary)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
# dictionary.filter_extremes(no_below=1, no_above=0.8)  THIS FILTERS EVERYTHING OUT
# print(dictionary)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]
# print(corpus)
%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xanen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


NameError: name 'input_texts' is not defined

NameError: name 'preprocess' is not defined

NameError: name 'tokenized_text' is not defined

NameError: name 'texts' is not defined

In [ ]:
lda.show_topics()


In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=15)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,1]
for i in topic_words:
    print([str(word[0]) for word in i])
    print()

In [ ]:
# print(corpus)
lsi = models.lsimodel.LsiModel(corpus, num_topics=5, id2word=dictionary, chunksize=10000)
topics_matrix = lsi.show_topics(formatted=False, num_words=15)
topics_matrix = np.array(topics_matrix)
topic_words = topics_matrix[:,1]
for i in topic_words:
    print([str(word[0]) for word in i])
    print()